The purpose of this project: 

ThreadTrail has requested to analyze data on visits to their website. The goal is to build a funnel in order to pinpoint the amount of people that continue to the next step of their purchase. (This project will focuses on t-shirts and no other piece of apparel.)

Findings:

83% of users who visited the page did not add a t-shirt to their cart,
35% percent of users who added a t-shirt to their cart did not checkout,
25% percent of users who made it to checkout  did not purchase a shirt,

The weakest part of the funnel is clearly getting a person who visited the site to add a tshirt to their cart. Once they've added a t-shirt to their cart it is fairly likely they end up purchasing it. A suggestion could be to make the add-to-cart button more prominent on the front page. We can also conduct A/B tests to analyze user behavior. This can help provide insights on how to optimize each stage of the funnel, this improving the converion rate.

In [80]:
import pandas as pd

Import all the files

In [81]:
visits = pd.read_csv('visits.csv',
                     parse_dates=[1])
cart = pd.read_csv('cart.csv',
                   parse_dates=[1])
                   
checkout = pd.read_csv('checkout.csv',
                       parse_dates=[1])
purchase = pd.read_csv('purchase.csv',
                       parse_dates=[1])

Step 1: Inspected the DataFrames using `print` and `head`

In [82]:
print(visits.head(5))
print(cart.head(5))
print(checkout.head(5))
print(purchase.head(5))

                                user_id          visit_time
0  943647ef-3682-4750-a2e1-918ba6f16188 2017-04-07 15:14:00
1  0c3a3dd0-fb64-4eac-bf84-ba069ce409f2 2017-01-26 14:24:00
2  6e0b2d60-4027-4d9a-babd-0e7d40859fb1 2017-08-20 08:23:00
3  6879527e-c5a6-4d14-b2da-50b85212b0ab 2017-11-04 18:15:00
4  a84327ff-5daa-4ba1-b789-d5b4caf81e96 2017-02-27 11:25:00
                                user_id           cart_time
0  2be90e7c-9cca-44e0-bcc5-124b945ff168 2017-11-07 20:45:00
1  4397f73f-1da3-4ab3-91af-762792e25973 2017-05-27 01:35:00
2  a9db3d4b-0a0a-4398-a55a-ebb2c7adf663 2017-03-04 10:38:00
3  b594862a-36c5-47d5-b818-6e9512b939b3 2017-09-27 08:22:00
4  a68a16e2-94f0-4ce8-8ce3-784af0bbb974 2017-07-26 15:48:00
                                user_id       checkout_time
0  d33bdc47-4afa-45bc-b4e4-dbe948e34c0d 2017-06-25 09:29:00
1  4ac186f0-9954-4fea-8a27-c081e428e34e 2017-04-07 20:11:00
2  3c9c78a7-124a-4b77-8d2e-e1926e011e7d 2017-07-13 11:38:00
3  89fe330a-8966-4756-8f7c-3bdbcd47279a 

Left merging visits and cart

In [83]:
visits_cart = visits.merge(cart, how = 'left')

visits_cart.head()

,user_id,visit_time,cart_time
0,943647ef-3682-4750-a2e1-918ba6f16188,2017-04-07 15:14:00,NaT
1,0c3a3dd0-fb64-4eac-bf84-ba069ce409f2,2017-01-26 14:24:00,2017-01-26 14:44:00
2,6e0b2d60-4027-4d9a-babd-0e7d40859fb1,2017-08-20 08:23:00,2017-08-20 08:31:00
3,6879527e-c5a6-4d14-b2da-50b85212b0ab,2017-11-04 18:15:00,NaT
4,a84327ff-5daa-4ba1-b789-d5b4caf81e96,2017-02-27 11:25:00,NaT


How long is `visits_cart`? We got 2000 visits within the time frame that the data was collected.

In [84]:
total_visits = len(visits_cart)

print(total_visits)

2000


How many timestamps are null for `cart_time`? Out of the 2000 visits, 348 customers proceeded to adding items to their carts and 1652 visitors added nothing to their carts.

In [85]:
nulls_in_cart_time = visits_cart['cart_time'].isnull().sum()
print(nulls_in_cart_time)
print(total_visits - nulls_in_cart_time)

1652
348


What percentage only visited? 82.6 percent of customers that visit do not proceed.

In [86]:
percent_only_visited = float(nulls_in_cart_time)/float(total_visits) * 100

print("Percent only visited:", percent_only_visited)

Percent only visited: 82.6


What percentage placed a t-shirt in their cart but did not checkout? 35% of customers that put a t-shirt in their cart proceeded to checkout. 

In [87]:
checkout_and_cart = cart.merge(checkout, how = 'left')

total_in_checkout_and_cart = len(checkout_and_cart)

nulls_in_checkout = checkout_and_cart['checkout_time'].isnull().sum()

percent_null_in_checkout = float(nulls_in_checkout)/float(total_in_checkout_and_cart)*100

print(checkout_and_cart.head())

print("Cart but not checkout:", percent_null_in_checkout)



                                user_id           cart_time  \
0  2be90e7c-9cca-44e0-bcc5-124b945ff168 2017-11-07 20:45:00   
1  4397f73f-1da3-4ab3-91af-762792e25973 2017-05-27 01:35:00   
2  a9db3d4b-0a0a-4398-a55a-ebb2c7adf663 2017-03-04 10:38:00   
3  b594862a-36c5-47d5-b818-6e9512b939b3 2017-09-27 08:22:00   
4  a68a16e2-94f0-4ce8-8ce3-784af0bbb974 2017-07-26 15:48:00   

        checkout_time  
0 2017-11-07 21:14:00  
1                 NaT  
2 2017-03-04 11:04:00  
3 2017-09-27 08:26:00  
4                 NaT  
Cart but not checkout: 35.05747126436782


Merge all df together.

In [88]:
all_data = visits_cart\
.merge(checkout_and_cart, how = 'left')\
.merge(purchase, how = 'left')
total_rows = all_data.shape[0]

print("Total number of rows in all_data:", total_rows)
print(all_data.head(5))

Total number of rows in all_data: 2108
                                user_id          visit_time  \
0  943647ef-3682-4750-a2e1-918ba6f16188 2017-04-07 15:14:00   
1  0c3a3dd0-fb64-4eac-bf84-ba069ce409f2 2017-01-26 14:24:00   
2  6e0b2d60-4027-4d9a-babd-0e7d40859fb1 2017-08-20 08:23:00   
3  6879527e-c5a6-4d14-b2da-50b85212b0ab 2017-11-04 18:15:00   
4  a84327ff-5daa-4ba1-b789-d5b4caf81e96 2017-02-27 11:25:00   

            cart_time       checkout_time       purchase_time  
0                 NaT                 NaT                 NaT  
1 2017-01-26 14:44:00 2017-01-26 14:54:00 2017-01-26 15:08:00  
2 2017-08-20 08:31:00                 NaT                 NaT  
3                 NaT                 NaT                 NaT  
4                 NaT                 NaT                 NaT  


24% of users who got to checkout but did not purchase

In [89]:
customer_checkout = all_data['checkout_time'].notnull().sum()
purchase_null_checkout_not_null = all_data['purchase_time'].isnull() & all_data['checkout_time'].notnull()
count_purchase_null_checkout_not_null = purchase_null_checkout_not_null.sum()
users_in_checkout_no_purchase = float(count_purchase_null_checkout_not_null)/float(customer_checkout)*100
print("Customers who made it to checkout:", customer_checkout)
print("Customers who did not purchase:", count_purchase_null_checkout_not_null)
print("Users who got to checkout but did not purchase:", users_in_checkout_no_purchase)

Customers who made it to checkout: 334
Customers who did not purchase: 82
Users who got to checkout but did not purchase: 24.550898203592812


Step 9: check each part of the funnel, let's print all 3 of them again

In [90]:
print("{} percent of users who visited the page did not add a t-shirt to their cart".format(round(percent_only_visited)))
print("{} percent of users who added a t-shirt to their cart did not checkout".format(round(percent_null_in_checkout)))
print("{} percent of users who made it to checkout  did not purchase a shirt".format(round(users_in_checkout_no_purchase)))


83 percent of users who visited the page did not add a t-shirt to their cart
35 percent of users who added a t-shirt to their cart did not checkout
25 percent of users who made it to checkout  did not purchase a shirt


*The weakest part of the funnel is clearly getting a person who visited the site to add a tshirt to their cart. Once they've added a t-shirt to their cart it is fairly likely they end up purchasing it. A suggestion could be to make the add-to-cart button more prominent on the front page.*